this OLS multiple linear regression implementation is fitted on Fisher's Iris data set.
We will try to classify flowers into 3 types based on their sepal length, sepal width,
petal length, petal length in cm. One class is linearly separable from the other 2;
the latter are NOT linearly separable from each other.
Since the focus of this code is the algorithm itself rather than the data,
we will spend no time analysing, or visualizing the data.



loading the data:

In [15]:
import numpy as np
from sklearn.datasets import load_iris

data = load_iris()
X = data['data']
predictor_num = len(data['feature_names'])
example_num = X.shape[0]

Y = data['target']
Y = Y[:, None]

Our X variable is a matrix consisting of $m$ examples and $n$ features. Each of the $m$ examples
corresponds to an out an entry in the response vector.

$X = \begin{bmatrix} x_{1,1} & x_{1,2} & \cdots & x_{1,n}\\
 x_{2,1} & x_{2,2} & \cdots & x_{2,n} \\
  x_{3,1} & x_{3,2} & \cdots & x_{3,n} \\
   \vdots & \vdots & \ddots & \vdots \\
\     x_{m,1} & x_{m,2} & \cdots & x_{m,n}  \end{bmatrix},
 Y = \begin{bmatrix} y_1 \\ y_2 \\ y_3\\ \vdots \\ y_n\end{bmatrix} $

In order to predict the response variable, we want to fit a line through our
data.

$Y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \cdots + \beta_nx_n$

In order for us to make the equation more compact we will represent it as the dot product of two matrices. First,
we will put all the coefficients in a vector. Then, we will put another vector consisting of
1s in the $X$ matrix; this will allow us to include the y-intercept term in the dot product. Thus, $X$ will become:

$X = \begin{bmatrix} 1& x_{1,1} & x_{1,2} & \cdots & x_{1,n}\\
 1 & x_{2,1} & x_{2,2} & \cdots & x_{2,n} \\
  1 & x_{3,1} & x_{3,2} & \cdots & x_{3,n} \\
   \vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_{m,1} & x_{m,2} & \cdots & x_{m,n}  \end{bmatrix}$

In [16]:
# adding another feature vector of value 1
X = np.hstack(((np.ones((example_num, 1))), X))

Finally, we can rewrite the previous equation as follows:

$Y = \beta X$

Neat! Now before we look for a way to get the values of the coefficients, we have to answer an important question.
How will we measure the error of our regression line? We have to find a way to measure the error in order for us to find
the coefficients that minimize it.
Since this is an ordinary least squares(OLS) algorithm,
we will take the average of the squared residuals.

$Cost(\beta) = \frac{1}{m} \sum_{i=1}^{m}(\hat y - y)^2$

Where $\hat y$ is the predicted response for a specific example.

Again, since we are working with vectors, there is a nicer way to write the error equation.

$Cost(\beta) = \frac{1}{m} (\hat Y - Y)^T(\hat Y - Y)$

We want the values of the coefficients inside $\beta$ that minimize this error function.
 Since our error equation is quadratic, we know that it has a global minima of $0$.
 That also means that the derivative of the cost function at that global minima is $0$.
  We can use differential calculus to find the value of $\beta$ that satisfies that.

  $\frac{\partial}{\partial \beta} Cost(\beta) = \frac{\partial}{\partial \beta}\frac{1}{m}[ (X\beta - Y)^T(X\beta - Y)]$

  Note that, $\hat Y = X\beta$.

  And since , $(A + B)^T = A^T + B^T$.we can simplify even further:

  $\begin{align}\frac{\partial}{\partial \beta} Cost(\beta)  &= \frac{\partial}{\partial \beta}\frac{1}{m} [((X\beta)^T - Y^T)(X\beta - Y)] \\
  &= \frac{\partial}{\partial \beta}\frac{1}{m} [(X\beta)^T(X\beta) - (X\beta)^TY - Y^T(X\beta) - Y^TY]
  \end{align}$

  W know that $X\beta$ and $Y$ are both Vectors. So, the property
  $A^TB = B^TA$ holds.

  $\begin{align}\frac{\partial}{\partial \beta} Cost(\beta)  &= \frac{\partial}{\partial \beta}\frac{1}{m}[ (X\beta)^T(X\beta) - 2(X\beta)^TY - Y^TY] \\
  &= \frac{1}{m}\frac{\partial}{\partial \beta}[(X\beta)^T(X\beta) - 2(X\beta)^TY] - \frac{\partial}{\partial \beta}Y^TY \\
  &= \frac{1}{m}\frac{\partial}{\partial \beta}[(X\beta)^T(X\beta) - 2(X\beta)^TY] - 0 \\
  &= \frac{1}{m}[\frac{\partial}{\partial \beta}[\beta^TX^TX\beta] -  \frac{\partial}{\partial \beta} [2\beta^TX^TY]]
  \end{align}$

  since we are trying to minimize the cost, we can set the LHS $\frac{\partial}{\partial \beta} Cost(\beta)$ to $0$.

  $\begin{align}&\frac{1}{m}[[\frac{\partial}{\partial \beta}\beta^TX^TX\beta] -  \frac{\partial}{\partial \beta} [2\beta^TX^TY]] = 0 \\
  &\frac{\partial}{\partial \beta}[\beta^TX^TX\beta] -  \frac{\partial}{\partial \beta} [2\beta^TX^TY] = 0 \\
  &\frac{\partial}{\partial \beta}[\beta^TX^TX\beta] =  \frac{\partial}{\partial \beta} [2\beta^TX^TY]
  \end{align}$

  Taking the derivative of the LHS:

  $\begin{align}
  &\frac{\partial}{\partial \beta}[\beta^TX^TX\beta]\\
  &= X^TX\frac{\partial}{\partial \beta}(\beta^T\beta) \\
  &= 2X^TX\beta
  \end{align}$

  Taking the derivative of the RHS:

  $\begin{align}
  &\frac{\partial}{\partial \beta} [2\beta^TX^TY]\\
  &= 2X^TY\frac{\partial}{\partial \beta} (\beta^T) \\
  &= 2X^TY
  \end{align}$

  Now that we computed our derivatives we can finally simplify the full equation to get $\beta$

  $\begin{align}
  &2X^TX\beta = 2X^TY \\
  &\beta = X^TY \\
  &\beta = (X^TX)^{-1}X^TY
  \end{align}$
  
  We finally derived the normal equation. We will use it to find the value of our $\beta$ vector
  that minimizes the error!

In [17]:
# calculating the slopes according to the normal equation
XTXInv = np.linalg.inv(np.dot(X.T,X))
XTY = np.dot(X.T, Y)
Beta = np.dot(XTXInv,XTY)

print(Beta)

[[ 0.18649525]
 [-0.11190585]
 [-0.04007949]
 [ 0.22864503]
 [ 0.60925205]]


 And now that we got our slopes, we can predict
 the response $Y$ with our linear regression equation:

$Y = X\beta$

In [18]:
Prediction = np.dot(X,Beta)

Now let's assess the quality of our fit!

First, we will compute the $R^2$ statistic to find out the proportion of variability in $Y$
explained by our model.

In [19]:
# the amount of variability inherent in
# the response before the regression is performed
TSS = float(np.dot((Y - np.mean(Y)).T,(Y - np.mean(Y))))

# the amount of unexplained variability
# left after the regression
RSS = float(np.dot((Prediction - Y).T,(Prediction - Y)))

# proportion of variability explained by our model
R2 = 1 - RSS/TSS

print(R2)

0.9303939218549564


Not bad! Our model explains 93% of the variability in $Y$
.

Now we will use the Residual standard error (RSE), adjusted to be unbiased, to find the
standard deviation of our error.

In [20]:
RSE = np.sqrt(RSS / (example_num - predictor_num - 1))

print(RSE)

0.2190985892792741


A standard deviation of error $\approx$ 0.22 is not bad considering
that $Y$ has a range $[0, 2]$

We could have used gradient descent to find our coefficients,
but it would have been computationally expensive compared to using the normal equation for such a small number of predictors.